# Demonstration of the v3 `PlantData` class

In [1]:
from copy import deepcopy
from pprint import pprint
from pathlib import Path

import numpy as np
import pandas as pd

from openoa.types.plant_v2 import PlantDataV3, PlantMetaData

## Load the data and create file mappings for later use

In [2]:
fpath = Path("data/la_haute_borne")
fn_scada = fpath / "la-haute-borne-data-2014-2015.csv"
fn_meter = fpath / "plant_data.csv"
fn_curtail = fpath / "plant_data.csv"
fn_reanalysis_merra2 = fpath / "merra2_la_haute_borne.csv"
fn_reanalysis_era5 = fpath / "era5_wind_la_haute_borne.csv"
fn_asset = fpath / "la-haute-borne_asset_table.csv"

scada = pd.read_csv(fn_scada)
meter = pd.read_csv(fn_meter)
curtail = pd.read_csv(fn_curtail)
reanalysis_era5 = pd.read_csv(fn_reanalysis_era5)
reanalysis_merra2 = pd.read_csv(fn_reanalysis_merra2)
asset = pd.read_csv(fn_asset)

latitude = 48.4497
longitude = 5.5896

yaml_meta = "data/plant_meta.yml"
json_meta = "data/plant_meta.json"

## TODO
 - [x] read data from spark, csv, pandas
 - [x] read metadata from json, yaml, dict, and pre-loaded object
 - [x] automatically calculate wind direction from u/v windspeed
 - [x] call planetos api if API key is provided
   - [ ] validate this works
 - [x] support flags for if csv/planetos/data object/etc
 - datetime column frequency checks
    - [ ] check against the provided metadata
    - [ ] validate against the analysis requirements
    - **note**: bring Lewis into this conversation on datetime & frequency validation, but is ok to use pandas for now
 - [x] expand metadata to contain plant-level identifiers (latitude, longitude)
 - check against the -25 namings and (likely) adopt that naming convention for the plant data
   - [ ] update internal column naming convention to the -25 schema (Eric/Lewis)
 - [x] map the input column names, and provide a method to provide them back as the original inputs
 - [x] get the 0 notebook working, or at least as a means to understand what will be required for refactoring
 - [ ] no failures for tower data as it's not used
 - [ ] none flag for raising warning, not error, for missing/bad data
 - [ ] flag to not raise an error for known missing data
 - [ ] metadata keyword argument for validate() to recreate `PlantMetaData`
     - allows for more flexibility in use cases, especially in the exploratory phase, or for changing analysis types
 - [ ] review the v3 todo workbook to stay on track with the rest of v3 development


## Create a dictionary of plant meta data 

**NOTE**: the datetime frequency checking is not in place, but the placeholder exists to implement it later

In [3]:
plant_meta = dict(
    latitude=latitude,
    longitude=longitude,
    scada=dict(
        time="Date_time",
        id="Wind_turbine_name",
        power="P_avg",
        windspeed="Ws_avg",
#         wtur_wspd="Ws_avg",  # TODO: adopt the -25 naming
        wind_direction="Wa_avg",
#         status="?",
        pitch="Ba_avg",
        temperature="Ot_avg",
        frequency="10T",
    ),
    meter=dict(
        time="time_utc",
        energy="net_energy_kwh",
    ),
    curtail=dict(
        time="time_utc",
        curtailment="curtailment_kwh",
        availability="availability_kwh",
        net_energy="net_energy_kwh",
        frequency="10T",
    ),
    reanalysis=dict(  # keys are informational/product-type, not pre-defined
        era5=dict(
            time="datetime",
            # windspeed="ws_100m",  # Commented out to demonstrate variable creation from base windspeed data
            windspeed_u="u_100",
            windspeed_v="v_100",
            temperature="t_2m",
            # density="dens_100m",  # Commented out to demonstrate variable creation from base windspeed data
            surface_pressure="surf_pres",
            frequency="H",
        ),
        merra2=dict(
            time="datetime",
            # windspeed="ws_50",  # Commented out to demonstrate variable creation from base windspeed data
            windspeed_u="u_50",
            windspeed_v="v_50",
            temperature="temp_10m",
            # density="dens_50",  # Commented out to demonstrate variable creation from base windspeed data
            surface_pressure="surface_pressure",
            frequency="H",
        )
    ),
    asset=dict(
        id="id",
        latitude="Latitude",
        longitude="Longitude",
        rated_power="Rated_power",
        hub_height="Hub_height_m",
        rotor_diameter="Rotor_diameter_m",
        elevation="elevation_m",
#         type="?",
    ),
)

# Recreate the YAML and JSON meta data objects as the dictionary above gets updated
import yaml
import json

with open(yaml_meta, "w") as f:
    yaml.safe_dump(plant_meta, f, default_flow_style=False)
    
with open(json_meta, "w") as f:
    json.dump(plant_meta, f, indent=4)

## Demonstrate the loading from YAML, JSON, and dictionary produce the exact same meta data

In [4]:
meta_from_dict = PlantMetaData.from_dict(plant_meta)
meta_from_json = PlantMetaData.from_json(json_meta)
meta_from_yaml = PlantMetaData.from_yaml(yaml_meta)
meta_from_dict == meta_from_json == meta_from_yaml, type(meta_from_dict)

(True, openoa.types.plant_v2.PlantMetaData)

## Show the PlantData capabilities

### Load from `DataFrame`s and a metadata dictionary

In [5]:
plant_from_data = PlantDataV3(
    metadata=meta_from_dict,
    scada=scada,
    meter=meter,
    curtail=curtail,
    reanalysis={"merra2": reanalysis_merra2, "era5": reanalysis_era5},  # preferred, and enable API pulling
    asset=asset,
    analysis_type="MonteCarloAEP",
)
type(plant_from_data)

openoa.types.plant_v2.PlantDataV3

### Show that "windspeed", "wind_direction", and "density" columns are all created from the core variables

In [6]:
plant_from_data.reanalysis["era5"].head()

,Unnamed: 0,time,windspeed_u,windspeed_v,temperature,surface_pressure,ws_100m,dens_100m,windspeed,wind_direction,density
0,0,1999-01-01 00:00:00+00:00,-4.456234,4.999991,277.496492,97020.784132,6.697606,1.216004,6.697606,138.291024,1.216004
1,1,1999-01-01 01:00:00+00:00,-5.006666,4.540059,277.330357,97002.617960,6.758612,1.216525,6.758612,132.201828,1.216525
2,2,1999-01-01 02:00:00+00:00,-5.101353,4.032729,276.968939,97035.490081,6.502823,1.218571,6.502823,128.327129,1.218571
3,3,1999-01-01 03:00:00+00:00,-5.787051,2.111372,276.721193,97021.649188,6.160183,1.219518,6.160183,110.044204,1.219518
4,4,1999-01-01 04:00:00+00:00,-6.349969,1.693571,276.291280,97005.213128,6.571932,1.221259,6.571932,104.933490,1.221259


In [7]:
plant_from_data.reanalysis["merra2"].head()

,Unnamed: 0,time,surface_pressure,surface_skin_temperature,u_10,v_10,windspeed_u,windspeed_v,temp_2m,temperature,u_850,v_850,temp_850,ws_50m,dens_50m,windspeed,wind_direction,density
0,0,1997-01-01 00:30:00+00:00,97353.040,258.85715,-5.517786,-3.350720,-7.167111,-4.020257,258.64264,258.54810,-10.273251,-0.722203,264.77643,8.217661,1.310616,8.217661,60.710568,1.311099
1,1,1997-01-01 01:30:00+00:00,97372.805,258.38250,-5.922551,-3.048721,-7.753224,-3.629969,258.16272,258.06815,-11.208241,-0.177881,265.06012,8.560909,1.313339,8.560909,64.911556,1.313824
2,2,1997-01-01 02:30:00+00:00,97371.945,258.06287,-5.918024,-2.925260,-7.747930,-3.496860,257.83590,257.74847,-11.152638,0.983774,265.51600,8.500497,1.315005,8.500497,65.709001,1.315454
3,3,1997-01-01 03:30:00+00:00,97354.710,257.95093,-5.587180,-3.093370,-7.363984,-3.779290,257.74982,257.68427,-9.815676,2.255801,266.19235,8.277155,1.315215,8.277155,62.832528,1.315552
4,4,1997-01-01 04:30:00+00:00,97348.410,257.99910,-5.471163,-3.099406,-7.257393,-3.835300,257.84320,257.80655,-8.338905,3.257462,266.77216,8.208488,1.314649,8.208488,62.144980,1.314838


### Show loading the data from file for both the meta data (JSON and YAML) and data (CSV)

In [8]:
plant_from_file1 = PlantDataV3(
    metadata=yaml_meta,
    scada=fn_scada,
    meter=fn_meter,
    curtail=fn_curtail,
    reanalysis={"merra2": fn_reanalysis_merra2, "era5": fn_reanalysis_era5},  # preferred, and enable API pulling
    asset=fn_asset,
    analysis_type="MonteCarloAEP"
)
type(plant_from_file1)

openoa.types.plant_v2.PlantDataV3

In [9]:
plant_from_file2 = PlantDataV3(
    metadata=json_meta,
    scada=fn_scada,
    meter=fn_meter,
    curtail=fn_curtail,
    reanalysis={"merra2": fn_reanalysis_merra2, "era5": fn_reanalysis_era5},  # preferred, and enable API pulling
    asset=fn_asset,
    analysis_type="MonteCarloAEP"
)
type(plant_from_file2)

openoa.types.plant_v2.PlantDataV3

### When updating the `analysis_type` to "all", note all the column data errors that are saved until the end of the validation

In [10]:
plant_from_data = PlantDataV3(
    metadata=meta_from_dict,
    scada=scada,
    meter=meter,
    curtail=curtail,
    reanalysis={"merra2": reanalysis_merra2, "era5": reanalysis_era5},  # preferred, and enable API pulling
    asset=asset,
    analysis_type="all"
)

ValueError: `scada` data is missing the following columns: ['status']
`meter` data is missing the following columns: ['power']
`tower` data is missing the following columns: ['time', 'id']
`asset` data is missing the following columns: ['type', 'id']
`status` data is missing the following columns: ['time', 'id', 'status_id', 'status_code', 'status_text']
`scada` data columns were of the wrong type: ['status']
`meter` data columns were of the wrong type: ['power']
`tower` data columns were of the wrong type: ['time', 'id']
`asset` data columns were of the wrong type: ['id', 'type']
`status` data columns were of the wrong type: ['time', 'id', 'status_id', 'status_code', 'status_text']

### Demonstrate changing a parameter (`analysis_type`) and revalidating with `PlantDataV3.validate()`

In [11]:
plant = deepcopy(plant_from_data)

In [12]:
plant.analysis_type = "all"
plant.validate()

ValueError: `scada` data is missing the following columns: ['Wind_turbine_name', 'P_avg', 'status', 'Ba_avg', 'Wa_avg', 'Date_time', 'Ot_avg', 'Ws_avg']
`meter` data is missing the following columns: ['net_energy_kwh', 'power', 'time_utc']
`tower` data is missing the following columns: ['time', 'id']
`asset` data is missing the following columns: ['Rated_power', 'Longitude', 'type', 'Latitude', 'id']
`status` data is missing the following columns: ['time', 'id', 'status_id', 'status_code', 'status_text']
`curtail` data is missing the following columns: ['net_energy_kwh', 'curtailment_kwh', 'time_utc', 'availability_kwh']
`reanalysis-merra2` data is missing the following columns: ['u_50', 'datetime', 'temp_10m', 'v_50']
`reanalysis-era5` data is missing the following columns: ['surf_pres', 'u_100', 't_2m', 'datetime', 'v_100']
`scada` data columns were of the wrong type: ['Date_time', 'Wind_turbine_name', 'P_avg', 'Ws_avg', 'Wa_avg', 'status', 'Ba_avg', 'Ot_avg']
`meter` data columns we

In [13]:
plant.analysis_type = "TurbineLongTermGrossEnergy"
plant.validate()

In [14]:
plant.analysis_type = "ElectricalLosses"
plant.validate()

#### Direct copy of the analysis requirements for easy referece

In [15]:
ANALYSIS_REQUIREMENTS = {
    "MonteCarloAEP": {
        "meter": {
            "columns": ["energy"],
            "freq": ("MS", "D", "H", "T"),
        },
        "curtail": {
            "columns": ["availability", "curtailment"],
            "freq": ("MS", "D", "H", "T"),
        },
        "reanalysis": {
            "columns": ["windspeed", "rho"],
            "conditional_columns": {
                "reg_temperature": ["temperature"],
                "reg_winddirection": ["windspeed_u", "windspeed_v"],
            },
        },
    },
    "TurbineLongTermGrossEnergy": {
        "scada": {
            "columns": ["id", "windspeed", "power"],  # TODO: wtur_W_avg vs energy_kwh ?
            "freq": ("D", "H", "T"),
        },
        "reanalysis": {
            "columns": ["windspeed", "wind_direction", "rho"],
        },
    },
    "ElectricalLosses": {
        "scada": {
            "columns": ["energy"],
            "freq": ("D", "H", "T"),
        },
        "meter": {
            "columns": ["energy"],
            "freq": ("MS", "D", "H", "T"),
        },
    },
}

### Show the updated column names and how to map them back to the original data

In [16]:
scada.columns.tolist()

['Wind_turbine_name',
 'Date_time',
 'Ba_avg',
 'P_avg',
 'Ws_avg',
 'Va_avg',
 'Ot_avg',
 'Ya_avg',
 'Wa_avg']

In [17]:
plant.scada.columns.tolist()

['id',
 'time',
 'pitch',
 'power',
 'windspeed',
 'Va_avg',
 'temperature',
 'Ya_avg',
 'wind_direction']

In [18]:
plant.update_column_names(to_original=True)
plant.scada.columns.tolist()

['Wind_turbine_name',
 'Date_time',
 'Ba_avg',
 'P_avg',
 'Ws_avg',
 'Va_avg',
 'Ot_avg',
 'Ya_avg',
 'Wa_avg']